In [71]:
import pandas as pd
import re
import csv
import demoji

In [72]:
pd.set_option('display.max_colwidth', None)

In [73]:
df = pd.read_csv('merged_data.csv')


In [74]:
df.shape

(2113, 4)

In [75]:
df.describe()

,Title,Content,URL,Comments
count,2113,78,2113,1949
unique,1843,78,2021,1914
top,Wow,"&#x200B;\n\nhttps://preview.redd.it/xzyck2xwtf1b1.png?width=500&format=png&auto=webp&s=824ae29301925464cd3dfef4c4eb58ae122dafe2\n\nThank you for reading our reviews!\n\nWe are a husband and wife whisky-reviewing duo in the Midwestern United States. We enjoy tasting and reviewing various types of whisky from around the world - Scotch, bourbon, rye, single malt, blends, you name it.\n\nPlease follow the link below to view our archive of whisky reviews:\n\n[https://docs.google.com/spreadsheets/d/1lvwcjXVvCkhv33XL-EP1k1Ek6f\_RK\_3kTvK0qykWMhI/edit?usp=sharing](https://docs.google.com/spreadsheets/d/1lvwcjXVvCkhv33XL-EP1k1Ek6f_RK_3kTvK0qykWMhI/edit?usp=sharing)",https://i.redd.it/eiph9fud3wu91.jpg,"Morris Inaugural Aged Port Barrel\n\nSingle malt whisky. 47.5% ABV. \n\nDistillery: Morris, Rutherglen, Victoria, Australia\n\nPrice: AU$165/US$106 for a 700ml bottle\n\nAge: NAS \n\nChill filtered: No\n\nBottled: unknown\n\nLimited edition: No\n\nMaturation: Finished in 10 Year Old Morris Port barrels\n\nBody: Maple syrup with ruby hues\n\nNose: Orange, brown sugar, dark chocolate\n\nPalate: Stone fruit\n\nFinish: Figs, blueberries, dark chocolate\n\nAugust 2023's offering from The Whisky Club is the Morris Inaugural Aged Port Barrel. The whisky is finished in 10 Year Old Morris Port barrels, and while they don't note where it spends most of the maturation it is presumably American oak barrels, likely re-fill. Side note Australia can't use the word 'Port' to describe their wine and must instead use the word 'Tawny', so I'm not sure if this is a typo, or if they're importing Portuguese wine casks. As Morris produce their own fortified wines and as such have access to some super fresh barrels, I suspect it's a typo.\n\nThe nose is undeniably fortified wine barrels and really packs a punch. Orange is dominant, while brown sugar and softer notes of dark chocolate sit behind. \n\nThe palate is rich and syrupy, suggesting the finish in fortified wine was anywhere up to 12 months, but likely closer to 6 months. This shows with the lack of depth, stone fruit is evident but there's no real depth of character behind this, even with a couple of drops of water. \n\nThe finish is chewy and lingering. Figs gives way to blueberries and a dark chocolate to round things out. \n\nOverall the Morris Aged Port Barrel is good without being great. Where Archie Rose's [A Whisky In Every Port](https://www.reddit.com/r/whisky/comments/ppa3j1/40th_whisky_review_12_australian_whisky_review/) had more depth and character the Morris starts well on the nose but fails to deliver thereafter, lacking the maturation time required to really create a knockout whisky. If you're a fan of whisky matured in port casks it's worth a look, however if you're new to this type of maturation there is better out there for the price point and depth. \n\n**Would I buy this to open in 10 years time:** \n\nNo, it's not unique enough.\n\n**Would I give this as a gift to a fellow whisky enthusiast:** \n\nLikely not, there's better out there for the price point.\n\n**Would I give this as a gift as an introductory whisky:** \n\nNo, it's not a great intro whisky.\n\n**Final Score: 75/100**\n\nRating Scale:\n\n0-50: Just bad.\n\n51-60: Shots only.\n\n61-70: Will do if there’s no better options.\n\n71-76: Average.\n\n77-82: Good (depending on price and availability, will probably \nbuy another bottle).\n\n83-87: Great (a cut above).\n\n88-92: Excellently Crafted.\n\n93-96: Superior.\n\n97-100: Whisky Nirvana.\n\nAll previous reviews can be found [here](https://www.reddit.com/r/DeppsDoesWhisky/comments/owah1r/all_reviews_by_udeppsdoeswhisky/).\n\nMy three favourites to date are \nMy three favourites reviewed to date are [Lagavulin 16](https://www.reddit.com/r/Scotch/comments/14voukw/64th_whisky_review_33rd_scotch_whisky_review/) (95), [Balvenie Doublewood 17](https://www.reddit.com/r/whisky/comments/loqg67/20th

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2113 entries, 0 to 2112
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     2113 non-null   object
 1   Content   78 non-null     object
 2   URL       2113 non-null   object
 3   Comments  1949 non-null   object
dtypes: object(4)
memory usage: 66.2+ KB


In [77]:
df.columns = [col.lower() for col in df.columns]

In [78]:
df.shape

(2113, 4)

In [79]:
def starts_with_review(text):
    if pd.isna(text):
        return False
    return text.strip().startswith("Review #")

df_filtered = df[df['title'].apply(starts_with_review)]

df_filtered.to_csv('filtered_data.csv', index=False, quoting=csv.QUOTE_ALL, lineterminator='\n')

In [80]:
df_filtered.shape

(1107, 4)

In [81]:
def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text)

def remove_emoji(text):
    return demoji.replace(text, "")

In [82]:
def clean_text(df):
    for col in df.columns:
        df[col] = df[col].astype(str).str.replace('\n', '', regex=False).replace('\r', '', regex=False)
        df[col] = df[col].astype(str).apply(remove_emoji)
        df[col] = df[col].astype(str).apply(remove_urls)
    return df

In [83]:
df_clean = clean_text(df_filtered.copy())

In [84]:
df_clean.to_csv('cleaned.csv', index=False, quoting=csv.QUOTE_ALL, lineterminator='\n')

In [85]:
df_clean.loc[[221]]

title content url  \
221  Review #55 - Blanton's Single Barrel Bourbon     nan       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [92]:
text = df_clean.loc[221,'comments']

start_keyword = "**Nose**:"
end_keyword = "**Palate**"
extracted_text = text[text.find(start_keyword) + len(start_keyword):text.find(end_keyword)].strip()

extracted_text

"Pretty classic profile here - sweetness, plenty of cherry and vanilla, and a decent amount of oak. As it opens up, that wood turns slightly more dry, but other sweet notes come forward, too: mild caramel, raw sugar, brown sugar, and some pie crust. There is a little bit of dusty corn grain and old worn leather, and at the end, we're finding some wood spice and cinnamon."

In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')

# text = "Pretty classic profile here - sweetness, plenty of cherry and vanilla, and a decent amount of oak. As it opens up, that wood turns slightly more dry, but other sweet notes come forward, too: mild caramel, raw sugar, brown sugar, and some pie crust. There is a little bit of dusty corn grain and old worn leather, and at the end, we're finding some wood spice and cinnamon."
text = "Lots of musty sherry, this is a bit of a sherry bomb. There is some mild oak in there, but that's musty as well, and some damp green earthiness accompanies it. Quite rich overall, and there are dried fruits of fig, orange, and then a bit of citrus. Not too sweet, and as this opens up further, the dried fruit turns to plum and raisin and grows stronger."
flavor_terms = set([
    "sweetness", "cherry", "vanilla", "oak", "dry", "caramel", "sugar", "brown sugar", "pie crust", 
    "corn", "grain", "leather", "spice", "cinnamon", "molasses", "maple", "wood", "citrus", 
    "apple", "pear", "peach", "tannin", "nutty", "honey", "smoky", "peat", "floral", "chocolate",
    "coffee", "toffee", "buttery", "creamy", "malty", "herbal", "earthy", "fruity", "nutmeg", "ginger",
    "orange", "lemon", "lime", "berry", "blackberry", "strawberry", "raspberry", "tropical", "banana",
    "pineapple", "coconut", "mango", "pepper", "spicy", "bitter","vanilla"
    # Add more relevant terms as needed
])

tokens = word_tokenize(text.lower())  # Lowercase for consistent matching
tagged_tokens = pos_tag(tokens)

# Filter for nouns and adjectives
keywords = [word for word, tag in tagged_tokens if tag in ('NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS')]

# Remove stopwords
stop_words = set(stopwords.words('english'))
keywords_filtered = [word for word in keywords if word not in stop_words]

# Filter against the flavor_terms list
flavor_keywords = [word for word in keywords_filtered if word in flavor_terms]

print(keywords_filtered)
print(set(flavor_keywords))


['lots', 'musty', 'sherry', 'bit', 'sherry', 'bomb', 'mild', 'oak', 'musty', 'green', 'earthiness', 'rich', 'overall', 'dried', 'fruits', 'fig', 'orange', 'bit', 'citrus', 'sweet', 'dried', 'fruit', 'stronger']
{'orange', 'oak', 'citrus'}
